<a href="https://colab.research.google.com/github/erdemustun/Dosyalar/blob/main/FiyatKanalBreakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --no-cache-dir git+https://github.com/rongardF/tvdatafeed.git
!pip install tradingview-screener
!pip install retrying


In [30]:
import numpy as np
import pandas as pd
import datetime
import time
import pytz
from tabulate import tabulate
pd.options.mode.chained_assignment = None
from tvDatafeed import TvDatafeed, Interval
tv = TvDatafeed()
from tradingview_screener import Query, Column
# from Tablo_RenderEt import render_mpl_table
# import send_to_telegram as tm #telegrama göndermek için
import linecache
import sys
import traceback
import subprocess # bash script komut gondermek
from retrying import retry
from tradingview_screener import Query, Column
from google.colab import files
import plotly.graph_objects as go
from scipy import stats

In [38]:
@retry(wait_fixed=4000, stop_max_attempt_number=3) # Hata olması halinde 4 sn bekleyip tekrar deneyecek, 3 kez deneyecek
def veri_getir(symbol,intervald):
    return tv.get_hist(symbol=symbol, exchange='BIST', interval=intervald, n_bars=365)

#Hataları ve hangi satırda olduğunu bulmak için
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print( 'Hata ({}, Satir: {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

# Zaman bilgisini alıp DataFrame'e ekleme (GMT+3)
time_zone = pytz.timezone('Europe/Istanbul')
dt = datetime.datetime.now(time_zone)
gun = dt.isoweekday()
saat = dt.hour
#dakika = dt.minute

In [21]:
#Hisseler = get_all_symbols(market='turkey') #Yurtdışı borsalar için  fon ve ba şka senteler geliyordu.
piyasa= 'turkey'
#piyasa= 'germany'
#piyasa= 'america'
options = ['stock']  #['stock', 'fund','dr']
#Burada tupple dönüyor
HisseAd_Sorgu = (Query().set_markets(f'{piyasa}')
                .select('name', 'type')
                .where(Column('type').isin(options) #(['stock', 'fund'])
                    )
                .limit(20000) #deault değeri 50 olduğu için hepsini getirmiyor.Onu aşmak için yüksek değer koydum.
                .order_by('name')
                .get_scanner_data())[1]

#HisseAd_Sorgu dataframe dönüştür
HisseAd = pd.DataFrame(HisseAd_Sorgu, columns=['ticker', 'name', 'type'])
#Listeye dönüştür
Hisseler = HisseAd['ticker'].tolist()

#Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]

# Dışarıdan eklemek istediğiniz endeks sembolleri
endeksler = ['XU030','XU100','XINSA','XBLSM','XUTEK','XGMYO','XHOLD','XFINK','XSGRT','XBANK','XUMAL','XILTM','XTRZM','XULAS','XELKT','XUHIZ','XMESY','XMANA','XTAST','XKMYA','XKAGT','XTEKS','XGIDA','XUSIN','XMADN']
Hisseler.extend(endeksler)
Hisseler = sorted(Hisseler)
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]

bist_hisseler=Hisseler

In [22]:

# Periyot belirleme
if gun == 7:  # Pazar
    periyot = 'H'  # Haftalık
    # if saat == 9:  # Sadece saat 10'da çalışsın
    #     periyot = 'H'  # Haftalık
    # else:
    #    #print("Pazar günü sadece saat 10'da çalışır.")
    #     sys.exit()
elif gun < 6:  # Pazartesi-Cuma
    if saat == 9:
        periyot = 'G'  # Günlük
    else:
        periyot = 'S'  # Saatlik
else:  # Cumartesi
    #print("Bu script Pazartesi-Cuma ve Pazar (saat 10'da) günleri çalışır.")
    sys.exit()


if periyot=='H':
    period='7y'
    intervald=Interval.in_weekly
    dosya='Haftalik'
elif periyot=='G':
    period='2y'
    intervald=Interval.in_daily
    dosya='Gunluk'
elif periyot=='S':
    period='1y'
    intervald=Interval.in_1_hour
    dosya='Saatlik'
elif periyot=='4S':
    period='1y'
    intervald=Interval.in_4_hour
    dosya='4Saatlik'
elif periyot=='D':
    period='15d'
    intervald=Interval.in_15_minute
    dosya='15dakikalık'
else:
    print('Hatalı girdi')

In [39]:
data=veri_getir('KCHOL',intervald)

In [40]:
# Tarih sütununu oluştur
data['Tarih'] = data.index

# Index'i sıfırla
data = data.reset_index(drop=True)

# OHLC sütunlarının adını ilk harfi büyük olacak şekilde değiştir
data = data.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'
})
data['symbol'] = data['symbol'].str.replace('BIST:', '', regex=False)


In [41]:
df=data.copy()

# Channel Break Out Detection

## Detect Pivots/Fractals

In [3]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0

    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].Low > df.iloc[i].Low:
            pivotLow=0
        if df.iloc[candle].High < df.iloc[i].High:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0


In [58]:
window=10 #Dip/Tepeyi mumun sagında/solunda kaç muma bakacağız. 3 orijinali,ne çok az ne çok fazla
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [59]:
def pointpos(x):
    if x['isPivot']==2:
        return x['Low']-1e-3
    elif x['isPivot']==1:
        return x['High']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [60]:
dfpl =df.copy() #df[0:100]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

## Detect Price Channel

In [47]:
def collect_channel(candle, backcandles, window):
    localdf = df[candle-backcandles-window:candle-window]
    localdf['isPivot'] = localdf.apply(lambda x: isPivot(x.name,window), axis=1)
    highs = localdf[localdf['isPivot']==1].High.values
    idxhighs = localdf[localdf['isPivot']==1].High.index
    lows = localdf[localdf['isPivot']==2].Low.values
    idxlows = localdf[localdf['isPivot']==2].Low.index

    if len(lows)>=2 and len(highs)>=2:
        sl_lows, interc_lows, r_value_l, _, _ = stats.linregress(idxlows,lows)
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)

        return(sl_lows, interc_lows, sl_highs, interc_highs, r_value_l**2, r_value_h**2)
    else:
        return(0,0,0,0,0,0)


In [48]:
candle = 75
backcandles = 40
window = 3

#dfpl = df[candle-backcandles-window-5:candle+200]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")

sl_lows, interc_lows, sl_highs, interc_highs, r_sq_l, r_sq_h = collect_channel(candle, backcandles, window)
print(r_sq_l, r_sq_h)
x = np.array(range(candle-backcandles-window, candle+1))
fig.add_trace(go.Scatter(x=x, y=sl_lows*x + interc_lows, mode='lines', name='lower slope'))
fig.add_trace(go.Scatter(x=x, y=sl_highs*x + interc_highs, mode='lines', name='max slope'))
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

0 0


## Detect Break Out

In [49]:
def isBreakOut(candle, backcandles, window):
    if (candle-backcandles-window)<0:
        return 0

    sl_lows, interc_lows, sl_highs, interc_highs, r_sq_l, r_sq_h = collect_channel(candle,
                                                                                   backcandles,
                                                                                   window)

    prev_idx = candle-1
    prev_high = df.iloc[candle-1].High
    prev_low = df.iloc[candle-1].Low
    prev_close = df.iloc[candle-1].Close

    curr_idx = candle
    curr_high = df.iloc[candle].High
    curr_low = df.iloc[candle].Low
    curr_close = df.iloc[candle].Close
    curr_open = df.iloc[candle].Open

    if ( prev_high > (sl_lows*prev_idx + interc_lows) and
        prev_close < (sl_lows*prev_idx + interc_lows) and
        curr_open < (sl_lows*curr_idx + interc_lows) and
        curr_close < (sl_lows*prev_idx + interc_lows)): #and r_sq_l > 0.9
        return 1

    elif ( prev_low < (sl_highs*prev_idx + interc_highs) and
        prev_close > (sl_highs*prev_idx + interc_highs) and
        curr_open > (sl_highs*curr_idx + interc_highs) and
        curr_close > (sl_highs*prev_idx + interc_highs)): #and r_sq_h > 0.9
        return 2

    else:
        return 0

In [51]:
def breakpointpos(x):
    if x['isBreakOut']==2:
        return x['Low']-3e-3
    elif x['isBreakOut']==1:
        return x['High']+3e-3
    else:
        return np.nan

candle = 75
backcandles = 40
window = 3
dfpl = df[candle-backcandles-window-5:candle+20]
dfpl["isBreakOut"] = [isBreakOut(candle, backcandles, window) for candle in dfpl.index]
dfpl['breakpointpos'] = dfpl.apply(lambda row: breakpointpos(row), axis=1)

In [53]:
candle = 73

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")

fig.add_scatter(x=dfpl.index, y=dfpl['breakpointpos'], mode="markers",
                marker=dict(size=8, color="Black"), marker_symbol="hexagram",
                name="Break out")

sl_lows, interc_lows, sl_highs, interc_highs, r_sq_l, r_sq_h = collect_channel(candle, backcandles, window)
print(r_sq_l, r_sq_h)
x = np.array(range(candle-backcandles-window, candle+1))
fig.add_trace(go.Scatter(x=x, y=sl_lows*x + interc_lows, mode='lines', name='lower slope'))
fig.add_trace(go.Scatter(x=x, y=sl_highs*x + interc_highs, mode='lines', name='max slope'))
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

1.0 1.0
